***
### Import of required libraries
***

In [ ]:
from geographiclib.geodesic import Geodesic
import pandas as pd
from tqdm.auto import tqdm

from traffic.core import Traffic

***
### Generation of SID classification dataset
***

In [ ]:
# Import of trajectories
t = Traffic.from_file("/mnt/beegfs/store/krum/MT/inputs/samax_trajs.parquet")


# Function to calculate bearing between two points
def get_bearing(lat1, lat2, long1, long2):
    brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)["azi1"]
    if brng < 0:
        brng = brng + 360
    return brng


# List to store bearings
ids = []
bearings = []

# For all flights, determine the bearing between Zurich airport and the last
# point of the flight and store it in the lists (ids and bearings)
for flight in tqdm(t):
    lat_ac_in = flight.data.iloc[0].latitude
    lon_ac_in = flight.data.iloc[0].longitude
    lat_ac_fin = flight.data.iloc[-1].latitude
    lon_ac_fin = flight.data.iloc[-1].longitude
    bearings.append(get_bearing(lat_ac_in, lat_ac_fin, lon_ac_in, lon_ac_fin))
    ids.append(flight.flight_id)

# Create a DataFrame with the flight ids and bearings
data = {"flight_id": ids, "bearing": bearings}
df_sid = pd.DataFrame(data)

# Assign the SID based on the bearing
df_sid["SID"] = "VEBIT"
df_sid.loc[df_sid["bearing"] < 210, "SID"] = "GERSA"
df_sid.loc[df_sid["bearing"] < 140, "SID"] = "DEGES"
df_sid.loc[df_sid["bearing"] < 75, "SID"] = "ZUE"

# Save dataframe as parquet file
df_sid.to_parquet("/mnt/beegfs/store/krum/MT/inputs/df_sid.parquet")